In [1]:
import nltk
import pandas as pd
import numpy as np
import json
from pprint import pprint
import plotly.express as px
import re
from spellchecker import SpellChecker
spell = SpellChecker()

# Cargar el archivo JSON en una variable Python
filename = 'your_posts_1.json'
with open(filename, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Convertir los datos JSON en un DataFrame de pandas
df = pd.json_normalize(data)



In [2]:


# Eliminar límite máximo de filas
pd.set_option('display.max_rows', None)

# Eliminar límite máximo de columnas
pd.set_option('display.max_columns', None)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4325 entries, 0 to 4324
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    4325 non-null   int64 
 1   attachments  3032 non-null   object
 2   data         4321 non-null   object
 3   title        3079 non-null   object
 4   tags         25 non-null     object
dtypes: int64(1), object(4)
memory usage: 169.1+ KB


In [4]:

# Convierte la columna 'timestamp' a un formato de fecha y hora
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [5]:



# Extrae el primer elemento de la columna 'attachments'
first_attachment = df.at[30, 'attachments']

# Imprime el valor con pprint para una mejor legibilidad
pprint(first_attachment)

[{'data': [{'external_context': {'url': 'http://www.filmaffinity.com/es/film546027.html'}}]}]


In [6]:
# Reemplaza los NaN por listas vacías en la columna 'attachments'
df['attachments'] = df['attachments'].apply(lambda x: x if isinstance(x, list) else [])

# Ahora puedes crear df_adjuntos sin el error de tipo
df_adjuntos = pd.DataFrame(df['attachments'].values.tolist(), columns=['attachments'])

In [7]:
x = df_adjuntos['attachments'].iloc[63]


In [8]:
for index, row in df_adjuntos.iterrows():
    attachment = row['attachments']
    if pd.isna(attachment) or not attachment or len(str(attachment)) <= 4:
        df_adjuntos.at[index, 'tipo'] = 'None'
        df_adjuntos.at[index, 'titulo'] = 'None'
        df_adjuntos.at[index, 'enlace'] = 'None'
    else:
        # Extraemos el primer valor de la lista que es un diccionario
        x0 = list(attachment.values())[0]
        # Extraemos el primer valor de x0 si x0 es una lista no vacía, de lo contrario None
        x1 = x0[0] if isinstance(x0, list) and x0 else None
        # Procedemos solo si x1 es un diccionario
        if isinstance(x1, dict) and x1:
            tipo = list(x1.keys())[0]
            # Extraemos el valor correspondiente a la clave tipo si está disponible
            x2 = x1.get(tipo)
            # Procedemos solo si x2 es un diccionario
            if isinstance(x2, dict) and x2:
                titulo = x2.get('title') or x2.get('name') or 'None'  # Verifica múltiples claves para el título
                enlace = x2.get('url', 'None')
            else:
                titulo = 'None'
                enlace = 'None'
        else:
            tipo = 'None'
            titulo = 'None'
            enlace = 'None'
        
        df_adjuntos.at[index, 'tipo'] = tipo
        df_adjuntos.at[index, 'titulo'] = titulo
        df_adjuntos.at[index, 'enlace'] = enlace

/var/folders/n4/hb64hxbn4bn6p9540q84tpcr0000gn/T/ipykernel_22236/1942196949.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'external_context' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_adjuntos.at[index, 'tipo'] = tipo
/var/folders/n4/hb64hxbn4bn6p9540q84tpcr0000gn/T/ipykernel_22236/1942196949.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Perfil de Spotify de Sigfrido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_adjuntos.at[index, 'titulo'] = titulo
/var/folders/n4/hb64hxbn4bn6p9540q84tpcr0000gn/T/ipykernel_22236/1942196949.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'http://open.spotify.com/user/lucas99' has dtype incompatible with float64, please explic

In [9]:
df_merge = pd.merge(df, df_adjuntos, left_index=True, right_index=True, how='left')

In [10]:
df_def = df_merge[['timestamp', 'data', 'tipo', 'titulo', 'enlace']]

In [11]:
# Crear un nuevo DataFrame con el índice y la columna 'data' de df_def
new_df = df_def[['data']].copy()

# Añadir una nueva columna 'count' que tiene el recuento del número de elementos en las listas de la columna 'data'
new_df['count'] = new_df['data'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Conservar el index original
new_df.reset_index(inplace=True)


In [12]:
# Inicializa la columna 'comentario' con valores por defecto None o puedes usar np.nan
new_df['comentario'] = None

# Itera a través de las filas con iterrows()
for index, row in new_df.iterrows():
    # Comprueba si la cuenta es 2
    if row['count'] >= 1:
        # Extrae el primer valor de la lista, que es un diccionario
        data_dict = row['data'][0]
        # Asume que el diccionario tiene solo un par clave-valor y obten el valor
        comentario = list(data_dict.values())[0]
        # Asigna el valor a la nueva columna 'comentario'
        new_df.at[index, 'comentario'] = comentario

In [13]:
df_publicaciones = pd.merge(df_def, new_df, left_index=True, right_index=True, how='left')

In [14]:
# Eliminar columnas no deseadas
df_publicaciones = df_publicaciones.drop(['data_x', 'data_y', 'index', 'count'], axis=1)

# Renombrar la columna 'timestamp' a 'fecha'
df_publicaciones = df_publicaciones.rename(columns={'timestamp': 'fecha'})

# Reordenar las columnas
df_publicaciones = df_publicaciones[['fecha', 'comentario', 'tipo', 'titulo', 'enlace']]

In [15]:


# Asumimos que 'df_publicaciones' es tu DataFrame y que 'fecha' es la columna que contiene las fechas en formato datetime
# Si 'fecha' no es un objeto datetime, descomenta y ejecuta la siguiente línea:
# df_publicaciones['fecha'] = pd.to_datetime(df_publicaciones['fecha'])

# Extraer año, mes, día y hora
df_publicaciones['año'] = df_publicaciones['fecha'].dt.year
df_publicaciones['mes'] = df_publicaciones['fecha'].dt.month
df_publicaciones['día'] = df_publicaciones['fecha'].dt.day
df_publicaciones['hora'] = df_publicaciones['fecha'].dt.hour

# Ahora df_publicaciones tiene columnas adicionales para año, mes, día y hora

In [16]:
import plotly.express as px

# Agrupa por año y mes, y cuenta las publicaciones por tipo
df_agrupado = df_publicaciones.groupby([df_publicaciones['fecha'].dt.to_period('M'), 'tipo']).size().reset_index(name='conteo')

# Crea un gráfico de barras
fig = px.bar(df_agrupado, x=df_agrupado['fecha'].dt.strftime('%Y-%m'), y='conteo', color='tipo',
             title='Número de publicaciones por tipo cada mes',
             labels={'conteo': 'Número de publicaciones', 'fecha': 'Fecha (Mes/Año)'})

# Mejora la apariencia del gráfico
fig.update_layout(
    xaxis_title='Fecha (Mes/Año)',
    yaxis_title='Número de Publicaciones',
    barmode='stack'  # Esto hace que las barras se apilen en lugar de agruparse
)

# Muestra el gráfico
fig.show()

In [17]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'df_publicaciones' es tu DataFrame y ya tiene la columna 'hora' extraída

# Agrupa por mes y hora para contar el número de publicaciones
grouped = df_publicaciones.groupby(['mes', 'hora']).size().reset_index(name='conteo')

# Calcula el total de publicaciones por mes
totals_per_month = grouped.groupby('mes')['conteo'].transform('sum')

# Calcula el porcentaje de publicaciones por hora dentro de cada mes
grouped['porcentaje'] = (grouped['conteo'] / totals_per_month) * 100

# Crea el gráfico de densidad de calor con porcentajes
fig = px.density_heatmap(grouped, x='mes', y='hora', z='porcentaje',
                         color_continuous_scale='RdYlGn',
                         title='Porcentaje de Publicaciones por Hora en Cada Mes')

# Muestra el gráfico
fig.show()

In [18]:
import pandas as pd
import plotly.express as px

# Asumiendo que 'df_publicaciones' es tu DataFrame y ya tiene las columnas necesarias

# Agrupa por mes, día y hora para contar el número de publicaciones
grouped = df_publicaciones.groupby(['mes', 'día', 'hora']).size().reset_index(name='conteo')

# Calcula el total de publicaciones por mes y día
totals_per_month_day = grouped.groupby(['mes', 'día'])['conteo'].transform('sum')

# Calcula el porcentaje de publicaciones por hora dentro de cada mes y día
grouped['porcentaje'] = (grouped['conteo'] / totals_per_month_day) * 100

# Crea el scatter plot 3D de burbujas
fig = px.scatter_3d(grouped, x='mes', y='día', z='hora', size='conteo', color='porcentaje',
                    color_continuous_scale='RdYlGn', title='Porcentaje de Publicaciones por Hora en Cada Mes y Día')

# Muestra el gráfico
fig.show()

In [19]:
# Definir el diccionario de reemplazo
reemplazos = {
    'None': 'Comentario o modificación',
    'text': 'Comentario o modificación',
    'media': 'docs subidos',
    'external_context': 'enlaces externos',  # Asegurarse de que la clave es correcta
    'event': 'evento',
    'place': 'lugar',
    "name": "Comentario con cita otro usuario"
}

# Reemplazar los valores en la columna 'tipo'
df_publicaciones['tipo'] = df_publicaciones['tipo'].replace(reemplazos)

# Verificar los cambios
print(df_publicaciones['tipo'].unique())

['enlaces externos' 'Comentario o modificación' 'docs subidos' 'lugar'
 'Comentario con cita otro usuario' 'evento']


In [20]:
import plotly.express as px

# Asumiendo que tienes una columna 'tipo' en df_publicaciones que clasifica cada publicación
fig = px.pie(df_publicaciones, names='tipo', title='Distribución de Tipos de Publicaciones')

fig.show()

In [21]:
import pandas as pd
import plotly.express as px
from urllib.parse import urlparse

# Asegúrate de que df_publicaciones está definido y contiene la columna 'enlace'

# Filtrar las filas donde la columna 'enlace' no está vacía y no es una cadena vacía
df_publicaciones_no_vacias = df_publicaciones[(df_publicaciones['enlace'].notna()) & (df_publicaciones['enlace'] != '')]

# Extraer el dominio de cada URL en la columna 'enlace'
df_publicaciones_no_vacias['dominio'] = df_publicaciones_no_vacias['enlace'].apply(lambda x: urlparse(x).netloc)

# Quitar 'www.' para mantener consistencia
df_publicaciones_no_vacias['dominio'] = df_publicaciones_no_vacias['dominio'].replace(r'^www\.', '', regex=True)

# Filtrar los dominios vacíos
df_publicaciones_no_vacias = df_publicaciones_no_vacias[df_publicaciones_no_vacias['dominio'] != '']

# Contar la frecuencia de cada dominio y obtener los 20 más frecuentes
conteo_dominios = df_publicaciones_no_vacias['dominio'].value_counts().nlargest(20).reset_index()
conteo_dominios.columns = ['dominio', 'conteo']

# Crear un gráfico de barras con los conteos de los dominios
fig = px.bar(conteo_dominios, x='dominio', y='conteo', title='Top 20 de URLs por Dominio')
fig.show()

In [22]:
# Filtrar los dominios vacíos y los que contienen 'youtube'
df_publicaciones_no_vacias_no_youtube = df_publicaciones_no_vacias[
    (df_publicaciones_no_vacias['dominio'] != '') & 
    (~df_publicaciones_no_vacias['dominio'].str.contains('youtube'))
]

# Contar la frecuencia de cada dominio y obtener los 50 más frecuentes
conteo_dominios_top50 = df_publicaciones_no_vacias_no_youtube['dominio'].value_counts().nlargest(50).reset_index()
conteo_dominios_top50.columns = ['dominio', 'conteo']

# Crear un gráfico de barras invertidas (horizontal) con los conteos de los dominios
fig = px.bar(conteo_dominios_top50, y='dominio', x='conteo', orientation='h', title='Top 50 de URLs por Dominio excluyendo vacíos y YouTube')
fig.show()

In [23]:
# Filtrar el DataFrame para obtener solo las filas con 'docs subidos'
docs_subidos_df = df_publicaciones[df_publicaciones['tipo'] == 'docs subidos']

# Contar las ocurrencias de cada 'titulo' único
conteo_titulos = docs_subidos_df['titulo'].value_counts()

# Crear el gráfico de queso
fig = px.pie(conteo_titulos, values=conteo_titulos.values, names=conteo_titulos.index, title='Distribución de Títulos para Documentos Subidos')

# Agregar el conteo total en las etiquetas
fig.update_traces(textinfo='percent+label+value')

# Mostrar el gráfico
fig.show()

In [24]:
# Filtrar para obtener solo las filas de 'Comentario o modificación' y donde 'comentario' no esté vacío
comentarios_df = df_publicaciones[(df_publicaciones['tipo'] == 'Comentario o modificación') & (df_publicaciones['comentario'].notnull())]

# Seleccionar las columnas de interés incluyendo las temporales ya existentes
comentarios_df = comentarios_df[['fecha', 'comentario', 'año', 'mes', 'día', 'hora']]

In [25]:

import numpy as np

# Asumiendo que 'comentarios_df' es tu DataFrame

# Función para verificar si el contenido es numérico
def es_numero(comentario):
    try:
        float(comentario) # Intenta convertir a float
        return True
    except ValueError:
        return False

# Aplicar la función para marcar los comentarios numéricos como NaN
comentarios_df['comentario'] = comentarios_df['comentario'].apply(lambda x: np.nan if es_numero(x) else x)

# Eliminar las filas con NaN en la columna 'comentario'
comentarios_df = comentarios_df.dropna(subset=['comentario'])


In [26]:
def normalize_text(text):
    words = re.findall(r'\w+', text.lower())
    corrected_text = []
    for word in words:
        corrected_word = spell.correction(word)
        # Asegúrate de que solo se añaden strings
        if isinstance(corrected_word, str):
            corrected_text.append(corrected_word)
        else:
            corrected_text.append(word)  # O puedes decidir omitir la palabra o manejar de otra manera
    return ' '.join(corrected_text)

comentarios_df['comentario_normalizado'] = comentarios_df['comentario'].apply(normalize_text)

In [27]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Asumimos que 'comentario' es la columna con los textos a analizar en tu DataFrame.
sia = SentimentIntensityAnalyzer()

# Creamos una nueva columna para la polaridad de los comentarios
comentarios_df['polaridad'] = comentarios_df['comentario'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

# Filtramos los valores para no incluir números o valores no deseados (si es necesario)
comentarios_df = comentarios_df[comentarios_df['comentario'].apply(lambda x: isinstance(x, str))]

# Creamos un nuevo DataFrame solo con los resultados del análisis de sentimiento
resultados_sentimiento = comentarios_df[['comentario', 'polaridad']].copy()

In [28]:
resultados_sentimiento


,comentario,polaridad
9,Me he tomado muchas pastillas seguidas. Es una...,-0.3736
21,dios esta cancion me tiene atrapado es perfecta,0.3400
28,no mola nada este disco,-0.2960
32,Malditas tardes neox...,0.0000
36,adiÃ³s QuÃ­mica!!,0.0000
37,adelante patata caliente!!!,0.0000
39,Placebo... pero Rosendo es droga dura!,0.0000
40,piscina...,0.0000
41,"-AquÃ­ corremos peligro\n-No importa, Machete ...",-0.2960
43,A Rosendo con lluvia!!!!!!!!!!!!,0.0000


In [30]:
sentimientos_def_df = comentarios_df.merge(resultados_sentimiento, left_index=True, right_index=True, suffixes=('', '_sentimiento'))


In [77]:
sentimientos_def_df.to_excel('sentimientos_def_df.xlsx', index=False)



In [78]:
sentimientos_def_df = pd.read_excel('sentimientos_def_df.xlsx')

In [79]:
sentimientos_def_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   fecha                   981 non-null    datetime64[ns]
 1   comentario              981 non-null    object        
 2   año                     981 non-null    int64         
 3   mes                     981 non-null    int64         
 4   día                     981 non-null    int64         
 5   hora                    981 non-null    int64         
 6   comentario_normalizado  979 non-null    object        
 7   polaridad               981 non-null    float64       
 8   comentario_sentimiento  981 non-null    object        
 9   polaridad_sentimiento   981 non-null    float64       
 10  polaridad_porcentaje    981 non-null    float64       
 11  polaridad_normalizada   981 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(4), object

In [80]:
import plotly.graph_objects as go

# Crear una nueva columna de polaridad en porcentaje
sentimientos_def_df['polaridad_porcentaje'] = sentimientos_def_df['polaridad_sentimiento'] * 100

# Crear el gráfico de línea temporal
fig2 = go.Figure()

# Agregar una capa de línea principal
fig2.add_trace(go.Scatter(
    x=sentimientos_def_df['fecha'],
    y=sentimientos_def_df['polaridad_porcentaje'],
    mode='lines',
    name='Polaridad (%)'
))

# Añadir una capa de marcadores de color
fig2.add_trace(go.Scatter(
    x=sentimientos_def_df['fecha'],
    y=sentimientos_def_df['polaridad_porcentaje'],
    mode='markers',
    marker=dict(
        size=8,
        color=sentimientos_def_df['polaridad_porcentaje'],
        colorscale='RdYlGn',  # Colores de rojo a verde
        colorbar=dict(title='Polaridad (%)'),
        cmin=-100,  # Valor mínimo en el rango de colores
        cmax=100,   # Valor máximo en el rango de colores
    ),
    showlegend=False  # No mostrar en la leyenda
))

# Personalizar el gráfico
fig2.update_layout(
    title='Polaridad del Sentimiento en Porcentaje a lo largo del tiempo',
    xaxis_title='Fecha',
    yaxis_title='Polaridad (%)',
    
)

# Mostrar el gráfico
fig2.show()


/Users/lucasfernandezmartinez/opt/anaconda3/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [59]:
import plotly.express as px

# Supongamos que tienes un DataFrame llamado sentimientos_def_df con la columna 'polaridad_sentimiento'

# Crear un gráfico de barras de frecuencia de polaridad
fig_frecuencia = px.histogram(
    sentimientos_def_df,
    x='polaridad_sentimiento',
    title='Frecuencia de Polaridad en Comentarios de Facebook',
    labels={'polaridad_sentimiento': 'Polaridad'},
    nbins=20,  # Ajusta el número de barras
    color_discrete_sequence=['skyblue']  # Color de las barras
)

# Personalizar el gráfico
fig_frecuencia.update_xaxes(title_text='Polaridad')
fig_frecuencia.update_yaxes(title_text='Frecuencia')
fig_frecuencia.show()


In [63]:
import plotly.express as px
import pandas as pd

# Supongamos que tienes un DataFrame llamado sentimientos_def_df con las columnas 'año' y 'polaridad_sentimiento'

# Calcular el promedio, mínimo y máximo de la polaridad por año
resumen_por_anio = sentimientos_def_df.groupby('año')['polaridad_sentimiento'].agg(['mean', 'min', 'max']).reset_index()

# Crear un gráfico de barras para mostrar promedio, mínimo y máximo por año
fig_promedio_min_max = px.bar(
    resumen_por_anio,
    x='año',
    y=['mean', 'min', 'max'],
    title='Polaridad de Comentarios de Facebook por Año (Promedio, Mínimo y Máximo)',
    labels={'value': 'Polaridad'},
    color_discrete_sequence=['skyblue', 'lightcoral', 'lightgreen'],  # Colores de las barras
)

# Personalizar el gráfico
fig_promedio_min_max.update_xaxes(title_text='Año')
fig_promedio_min_max.update_yaxes(title_text='Polaridad')
fig_promedio_min_max.show()


In [65]:
import plotly.express as px
import pandas as pd

# Supongamos que tienes un DataFrame llamado sentimientos_def_df con las columnas 'hora' y 'polaridad_sentimiento'

# Calcular el promedio, mínimo y máximo de la polaridad por hora
resumen_por_hora = sentimientos_def_df.groupby('hora')['polaridad_sentimiento'].agg(['mean', 'min', 'max']).reset_index()

# Crear un gráfico de barras para mostrar el promedio, mínimo y máximo por hora
fig_variacion_hora = px.bar(
    resumen_por_hora,
    x='hora',
    y=['mean', 'min', 'max'],
    title='Variación de la Polaridad de Comentarios de Facebook por Hora del Día',
    labels={'value': 'Polaridad'},
    color_discrete_sequence=['skyblue', 'lightcoral', 'lightgreen'],  # Colores de las barras
)

# Personalizar el gráfico
fig_variacion_hora.update_xaxes(title_text='Hora del Día')
fig_variacion_hora.update_yaxes(title_text='Polaridad')
fig_variacion_hora.show()


In [66]:
import plotly.express as px
import pandas as pd

# Supongamos que tienes un DataFrame llamado sentimientos_def_df con las columnas 'mes' y 'polaridad_sentimiento'

# Calcular el promedio, mínimo y máximo de la polaridad por mes
resumen_por_mes = sentimientos_def_df.groupby('mes')['polaridad_sentimiento'].agg(['mean', 'min', 'max']).reset_index()

# Crear un gráfico de barras para mostrar el promedio, mínimo y máximo por mes
fig_variacion_mes = px.bar(
    resumen_por_mes,
    x='mes',
    y=['mean', 'min', 'max'],
    title='Variación de la Polaridad de Comentarios de Facebook por Mes',
    labels={'value': 'Polaridad'},
    color_discrete_sequence=['skyblue', 'lightcoral', 'lightgreen'],  # Colores de las barras
)

# Personalizar el gráfico
fig_variacion_mes.update_xaxes(title_text='Mes')
fig_variacion_mes.update_yaxes(title_text='Polaridad')
fig_variacion_mes.show()


In [75]:
import plotly.express as px
import pandas as pd

# Supongamos que tienes un DataFrame llamado sentimientos_def_df con las columnas 'fecha', 'hora' y 'polaridad_sentimiento'

# Normalizar la columna de polaridad_sentimiento para que esté en el rango [0, 1]
min_polaridad = sentimientos_def_df['polaridad_sentimiento'].min()
max_polaridad = sentimientos_def_df['polaridad_sentimiento'].max()
sentimientos_def_df['polaridad_normalizada'] = (sentimientos_def_df['polaridad_sentimiento'] - min_polaridad) / (max_polaridad - min_polaridad)

# Crear un gráfico de dispersión 3D con la polaridad normalizada como tamaño y color
fig = px.scatter_3d(
    sentimientos_def_df,
    x='fecha',
    y='hora',
    z='polaridad_sentimiento',
    size='polaridad_normalizada',  # Usar la polaridad normalizada como tamaño
    color='polaridad_sentimiento',  # Color basado en la polaridad original
    color_continuous_scale=['red', 'green'],  # Rojo para negativo, verde para positivo
    labels={'polaridad_sentimiento': 'Polaridad'},
    title='Gráfico de Dispersión 3D de Polaridad vs. Fecha y Hora',
)

# Personalizar el gráfico
fig.update_layout(
    scene=dict(
        zaxis_title='Polaridad',
        xaxis_title='Fecha',
        yaxis_title='Hora',
    ),
    width=1000,  # Ancho personalizado
    height=600,  # Alto personalizado
    margin=dict(l=0, r=0, b=0, t=40),  # Personalizar los márgenes
)

fig.show()


/Users/lucasfernandezmartinez/opt/anaconda3/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

